In [1]:
import pandas as pd
import numpy as np


In [2]:
data = pd.read_csv('/content/train_LZdllcl.csv');

In [3]:
data.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [4]:
data.shape

(54808, 14)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54808 entries, 0 to 54807
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   employee_id           54808 non-null  int64  
 1   department            54808 non-null  object 
 2   region                54808 non-null  object 
 3   education             52399 non-null  object 
 4   gender                54808 non-null  object 
 5   recruitment_channel   54808 non-null  object 
 6   no_of_trainings       54808 non-null  int64  
 7   age                   54808 non-null  int64  
 8   previous_year_rating  50684 non-null  float64
 9   length_of_service     54808 non-null  int64  
 10  KPIs_met >80%         54808 non-null  int64  
 11  awards_won?           54808 non-null  int64  
 12  avg_training_score    54808 non-null  int64  
 13  is_promoted           54808 non-null  int64  
dtypes: float64(1), int64(8), object(5)
memory usage: 5.9+ MB


In [6]:
data.isnull().sum()

employee_id                0
department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
dtype: int64

In [7]:
#Data Pre-proprocessing

In [8]:
#remove unwanted columns
data.drop(['region','gender','recruitment_channel'], axis=1, inplace = True)

In [9]:
# Encode categorical variables using label encoding
from sklearn.preprocessing import LabelEncoder


In [10]:
label_encoder = LabelEncoder()

In [11]:
data['department'] = label_encoder.fit_transform(data['department'])
data['education'] = label_encoder.fit_transform(data['education'])

In [12]:
data['department'].unique()

array([7, 4, 8, 0, 6, 5, 1, 2, 3])

In [13]:
data['education'].unique()

array([2, 0, 3, 1])

In [14]:
# filling data with mode
data['education'] = data['education'].fillna(data['education'].mode()[0])

In [15]:
data.isnull().sum()

employee_id                0
department                 0
education                  0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
dtype: int64

In [16]:
data["previous_year_rating"].nunique()

5

In [17]:
import matplotlib.pyplot as plt
import seaborn as sns


In [18]:
data['previous_year_rating']=data['previous_year_rating'].fillna(data['previous_year_rating'].mode()[0])

In [19]:
data.isnull().sum()

employee_id             0
department              0
education               0
no_of_trainings         0
age                     0
previous_year_rating    0
length_of_service       0
KPIs_met >80%           0
awards_won?             0
avg_training_score      0
is_promoted             0
dtype: int64

In [20]:
data.head()

,employee_id,department,education,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,7,2,1,35,5.0,8,1,0,49,0
1,65141,4,0,1,30,5.0,4,0,0,60,0
2,7513,7,0,1,34,3.0,7,0,0,50,0
3,2542,7,0,2,39,1.0,10,0,0,50,0
4,48945,8,0,1,45,3.0,2,0,0,73,0


In [21]:

for i in ['age', 'length_of_service']:
    q25, q75 = np.percentile(data[i], [25, 75])
    iqr = q75 - q25

    lower_bound = q25 - iqr * 1.5
    upper_bound = q75 + iqr * 1.5

    data.loc[data[i] < lower_bound, i] = lower_bound
    data.loc[data[i] > upper_bound, i] = upper_bound



In [22]:
for i in ['age', 'length_of_service']:
    q25, q75 = np.percentile(data[i], [25, 75])
    iqr = q75 - q25

    lower_bound = q25 - iqr * 1.5
    upper_bound = q75 + iqr * 1.5

    data.loc[data[i] < lower_bound, i] = lower_bound
    data.loc[data[i] > upper_bound, i] = upper_bound


In [23]:
# Split train into X_train (Features) and y_train (Target) :
x=data.drop(['employee_id','is_promoted'],axis=1)
y=data['is_promoted']

In [24]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [25]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

# Random Forest
rfc = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=0)
rfc.fit(x_train, y_train)

rfc_pred= rfc.predict(x_test)

# XGBoost
xgb = XGBClassifier(max_depth=5,learning_rate=0.3,)
xgb.fit(x_train, y_train)

xgb_pred= xgb.predict(x_test)

In [26]:
from sklearn.metrics import accuracy_score, f1_score

print("RF Accuracy:", accuracy_score(rfc_pred, y_test))
print("RF F1 Score:", f1_score(rfc_pred, y_test))
print("XGBoost Accuracy:", accuracy_score(xgb_pred, y_test))
print("XGBoost F1 Score:", f1_score(xgb_pred, y_test))

RF Accuracy: 0.9328589673417259
RF F1 Score: 0.4666666666666666
XGBoost Accuracy: 0.945995256340084
XGBoost F1 Score: 0.4965986394557824


In [27]:
#predicting the output using test file

In [28]:
test_data = pd.read_csv('/content/test_2umaH9m.csv')

In [29]:
test_data.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,NaN,1,1,0,77
1,74430,HR,region_4,Bachelor's,f,other,1,31,3.0,5,0,0,51
2,72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.0,4,0,0,47
3,38562,Procurement,region_2,Bachelor's,f,other,3,31,2.0,9,0,0,65
4,64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.0,7,0,0,61


In [30]:
#Pre processing using same step
test_data.drop(['region','gender','recruitment_channel'], axis=1, inplace = True)

# Encode the 'department' column
test_data['department'] = label_encoder.fit_transform(test_data['department'])
# Encode the 'education' column
test_data['education'] = label_encoder.fit_transform(test_data['education'])

In [31]:
test_data.isna().sum()

employee_id                0
department                 0
education                  0
no_of_trainings            0
age                        0
previous_year_rating    1812
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
dtype: int64

In [32]:
test_data['previous_year_rating']=test_data['previous_year_rating'].fillna(test_data['previous_year_rating'].mode()[0])

In [45]:
test_data.columns

Index(['employee_id', 'department', 'education', 'no_of_trainings', 'age',
       'previous_year_rating', 'length_of_service', 'KPIs_met >80%',
       'awards_won?', 'avg_training_score'],
      dtype='object')

In [46]:
x_test=test_data.drop(['employee_id'],axis=1)
predictions = xgb.predict(x_test) # best trained model

In [47]:
# Create a DataFrame with employee_id and corresponding predictions
predictions_data = pd.DataFrame({'employee_id': test_data['employee_id'], 'is_promoted': predictions})
predictions_data.to_csv('/content/predictions.csv', index=False)